<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/tkyang/%EB%AC%B8%EC%84%9C%EC%9A%94%EC%95%BDTextRank%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

docs = '딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나.'

# 문장 토큰화
docs_token = docs.split('.')
sent1 = docs_token[0]
sent2 = docs_token[1]
sent3 = docs_token[2]
sent4 = docs_token[3]

# 단어 토큰화
sent1_token = sent1.split()
sent2_token = sent2.split()
sent3_token = sent3.split()
sent4_token = sent4.split()

# docs_token.pop()
# print(docs_token)

docs_t = [sent1_token, sent2_token, sent3_token, sent4_token]
docs_t

[['딸기', '바나나', '사과', '파인애플', '수박'],
 ['바나나', '사과', '딸기', '포도'],
 ['복숭아', '수박'],
 ['파인애플', '사과', '딸기', '바나나']]

In [2]:
trg = np.zeros((len(docs_t), len(docs_t)), dtype=float)
trg

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [3]:
# 문장간 유사도(자카드 유사도)
def simil(token1, token2):
    union = set(token1).union(set(token2))
    intersection = set(token1).intersection(set(token2))

    return len(intersection)/len(union)

# textrank 그래프 생성
for i in range(len(docs_t)):
    for j in range(len(docs_t)):
        trg[i, j] = simil(docs_t[i], docs_t[j])
    trg[i][i] = 0
trg

array([[0.        , 0.5       , 0.16666667, 0.8       ],
       [0.5       , 0.        , 0.        , 0.6       ],
       [0.16666667, 0.        , 0.        , 0.        ],
       [0.8       , 0.6       , 0.        , 0.        ]])

In [4]:
# 엣지 가중치
edge = trg.copy()
for i in range(len(edge)):
    sum = np.sum(edge[i])
    for j in range(len(edge)):
        edge[i,j] = edge[i][j]/sum
edge

array([[0.        , 0.34090909, 0.11363636, 0.54545455],
       [0.45454545, 0.        , 0.        , 0.54545455],
       [1.        , 0.        , 0.        , 0.        ],
       [0.57142857, 0.42857143, 0.        , 0.        ]])

In [5]:
# score 계산
def cal_score(matrix):
    d = 0.85
    score = []
    for i in range(len(matrix)):
        score.append((1-d) + d*(np.sum(matrix.T[i]))) # 열 단위로 합해주기 위한 Transpose
    return score

scores = cal_score(trg)
scores

[1.396666666666667, 1.085, 0.2916666666666667, 1.3399999999999999]

In [7]:
max_iter = 10

for iter in range(max_iter):
    for i in range(len(trg)):
        for w in range(len(trg)):
            trg[i, w] = scores[i]*edge[i][w]
    
    
    scores = cal_score(trg)
    
    print(trg)
    print('Score :',scores)
    print('#'*80)

[[0.         0.47613636 0.15871212 0.76181818]
 [0.49318182 0.         0.         0.59181818]
 [0.29166667 0.         0.         0.        ]
 [0.76571429 0.57428571 0.         0.        ]]
Score : [1.4679783549783552, 1.042858766233766, 0.28490530303030304, 1.3005909090909094]
################################################################################
[[0.         0.50044717 0.16681572 0.80071547]
 [0.47402671 0.         0.         0.56883205]
 [0.2849053  0.         0.         0.        ]
 [0.74319481 0.5573961  0.         0.        ]]
Score : [1.4268077971271151, 1.049166779811098, 0.2917933638331366, 1.3141153925619835]
################################################################################
[[0.         0.48641175 0.16213725 0.7782588 ]
 [0.47689399 0.         0.         0.57227279]
 [0.29179336 0.         0.         0.        ]
 [0.75092308 0.56319231 0.         0.        ]]
Score : [1.4416688707023004, 1.0421634511008102, 0.28781666222250546, 1.297951849307717]
#####

In [8]:
# threshold = 
# before_score = score.copy()
# if np.sum(before_score - score) < threshold:
#     break